## **Evaluation Code**

In [ ]:
BASE_DIR = '/content/drive/MyDrive/BrainTumorClassification'
TRAIN_DIR = f'{BASE_DIR}/data/Training'
TEST_DIR = f'{BASE_DIR}/data/Testing'

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.preprocessing import image
import pandas as pd
import os
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import random

class ModelEvaluator:
    def __init__(self, model, history=None, test_gen=None, class_names=None,
                 input_size=(224, 224), preprocess_fn=None, last_conv_layer=None):
        self.model = model
        self.history = history
        self.test_gen = test_gen
        self.class_names = class_names
        self.input_size = input_size
        self.preprocess_fn = preprocess_fn  # Optional preprocessing for Grad-CAM
        self.last_conv_layer = last_conv_layer

        # Ground truth and predictions
        self.y_true = self.test_gen.classes
        self.y_pred_probs = self.model.predict(self.test_gen, verbose=0)
        self.y_pred = np.argmax(self.y_pred_probs, axis=1)

        # Feature extraction for t-SNE (from second-to-last layer)
        try:
            feature_extractor = tf.keras.Model(
                inputs=self.model.input,
                outputs=self.model.layers[-2].output  # Adjust if necessary
            )
            self.features = feature_extractor.predict(self.test_gen, verbose=0)
            self.feature_labels = self.y_true
        except Exception as e:
            print(f"[Warning] Feature extraction failed: {e}")
            self.features = None
            self.feature_labels = None
        try:
            self.x_test, _ = extract_data_from_generator(self.test_gen)
        except Exception as e:
            print(f"[Warning] Failed to extract x_test: {e}")
            self.x_test = None



    def plot_training_curves(self):
        acc = self.history.history['accuracy']
        val_acc = self.history.history['val_accuracy']
        loss = self.history.history['loss']
        val_loss = self.history.history['val_loss']
        epochs = range(1, len(acc) + 1)

        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(epochs, acc, 'bo-', label='Train Acc')
        plt.plot(epochs, val_acc, 'ro-', label='Val Acc')
        plt.title('Accuracy'); plt.xlabel('Epochs'); plt.ylabel('Accuracy'); plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(epochs, loss, 'bo-', label='Train Loss')
        plt.plot(epochs, val_loss, 'ro-', label='Val Loss')
        plt.title('Loss'); plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.legend()

        plt.tight_layout(); plt.show()

    def plot_confusion_matrix(self, normalize=True):
        cm = confusion_matrix(self.y_true, self.y_pred)
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1, keepdims=True)

        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='.2f' if normalize else 'd',
                    xticklabels=self.class_names, yticklabels=self.class_names,
                    cmap='YlGnBu', square=True, linewidths=0.5)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted'); plt.ylabel('Actual')
        plt.tight_layout(); plt.show()

    def plot_roc_curves(self):
        y_true = self.y_true
        y_pred_probs = self.y_pred_probs
        class_names = self.class_names

        y_true_bin = label_binarize(y_true, classes=range(len(class_names)))

        plt.figure(figsize=(8, 6))
        for i in range(len(class_names)):
            fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f"{class_names[i]} (AUC = {roc_auc:.2f})")

        plt.plot([0, 1], [0, 1], 'k--', label='Chance')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves: One-vs-Rest')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()



    def plot_tsne(self):
        tsne = TSNE(n_components=2, random_state=42)
        reduced = tsne.fit_transform(self.features)

        sns.scatterplot(
            x=reduced[:, 0],
            y=reduced[:, 1],
            hue=[self.class_names[i] for i in self.feature_labels],
            palette='tab10'
        )
        plt.title('T-SNE Visualization of CNN Features')
        plt.tight_layout()
        plt.show()



    def plot_confidence_histogram(self, bins=10):
        confidences = np.max(self.y_pred_probs, axis=1)
        correct = self.y_pred == self.y_true
        incorrect = ~correct

        plt.figure(figsize=(8, 5))
        plt.hist(confidences[correct], bins=bins, alpha=0.6, label='Correct', color='green')
        plt.hist(confidences[incorrect], bins=bins, alpha=0.6, label='Incorrect', color='red')
        plt.xlabel('Prediction Confidence')
        plt.ylabel('Number of Samples')
        plt.title('Confidence Distribution of Model Predictions')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()


    def show_classification_report(self):
        report = classification_report(self.y_true, self.y_pred, target_names=self.class_names)
        print(report)

    def plot_per_class_accuracy(self):
        accs = []
        for i, name in enumerate(self.class_names):
            mask = (self.y_true == i)
            acc = accuracy_score(self.y_true[mask], self.y_pred[mask])
            accs.append(acc * 100)

        plt.figure(figsize=(8, 5))
        plt.bar(self.class_names, accs, color='orchid')
        plt.title('Per-Class Accuracy')
        plt.ylabel('Accuracy (%)'); plt.ylim(0, 100)
        plt.grid(axis='y', linestyle='--', alpha=0.5)
        plt.tight_layout(); plt.show()

    def show_misclassified_images(self, num_samples=9):

        if self.x_test is None:
            print("[Error] x_test not available. Ensure data was extracted properly.")
            return

        errors = [i for i in range(len(self.y_true)) if self.y_pred[i] != self.y_true[i]]
        if not errors:
            print("✅ No misclassifications found!")
            return

        selected = random.sample(errors, min(num_samples, len(errors)))

        plt.figure(figsize=(10, 10))
        for i, idx in enumerate(selected):
            plt.subplot(3, 3, i + 1)
            plt.imshow(self.x_test[idx])
            plt.title(f"True: {self.class_names[self.y_true[idx]]}\nPred: {self.class_names[self.y_pred[idx]]}")
            plt.axis('off')
        plt.tight_layout()
        plt.show()

    def gradcam_visualization(self, img_path):
        if not self.last_conv_layer:
            raise ValueError("You must provide last_conv_layer name in the constructor for Grad-CAM.")

        img = tf.keras.utils.load_img(img_path, target_size=self.input_size)
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.expand_dims(img_array, axis=0)

        if self.preprocess_fn:
          img_array = self.preprocess_fn(tf.cast(img_array, tf.float32))

        grad_model = tf.keras.models.Model(
            [self.model.inputs], [self.model.get_layer(self.last_conv_layer).output, self.model.output]
        )

        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(img_array)
            pred_index = tf.argmax(predictions[0])
            class_channel = predictions[:, pred_index]

        grads = tape.gradient(class_channel, conv_outputs)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        conv_outputs = conv_outputs[0]
        heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

        heatmap = cv2.resize(heatmap.numpy(), self.input_size)
        heatmap = np.uint8(255 * heatmap)
        heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

        superimposed = cv2.addWeighted(np.array(img), 0.6, heatmap_color, 0.4, 0)

        plt.figure(figsize=(6, 6))
        plt.imshow(superimposed.astype('uint8'))
        plt.title(f'Grad-CAM: {self.class_names[int(pred_index)]}')
        plt.axis('off')
        plt.tight_layout(); plt.show()


In [2]:
def gradcam_samples_by_class(evaluator, root_dir= TEST_DIR , max_examples=1):
    print("🔍 Sampling Grad-CAMs: 1 correct + 1 incorrect per class\n")

    # Invert label index for readability
    class_idx_to_label = {v: k for k, v in evaluator.test_gen.class_indices.items()}

    # Create prediction dataframe
    df = pd.DataFrame({
        'filename': evaluator.test_gen.filenames,
        'true': evaluator.y_true,
        'pred': evaluator.y_pred
    })

    df['true_label'] = df['true'].map(class_idx_to_label)
    df['pred_label'] = df['pred'].map(class_idx_to_label)
    df['correct'] = df['true'] == df['pred']

    # Collect one correct and incorrect example per class
    for class_idx, class_name in class_idx_to_label.items():
        correct = df[(df['true'] == class_idx) & (df['correct'])]
        incorrect = df[(df['true'] == class_idx) & (~df['correct'])]

        if not correct.empty:
            for _, row in correct.sample(min(max_examples, len(correct))).iterrows():
                print(f"✅ {class_name} - Correct")
                path = os.path.join(root_dir, row['filename'])
                evaluator.gradcam_visualization(path)

        if not incorrect.empty:
            for _, row in incorrect.sample(min(max_examples, len(incorrect))).iterrows():
                print(f"❌ {class_name} - Incorrect (Pred: {row['pred_label']})")
                path = os.path.join(root_dir, row['filename'])
                evaluator.gradcam_visualization(path)


NameError: name 'TEST_DIR' is not defined

In [ ]:
def show_misclassified_images(self, num_samples=9):
    import random

    if self.x_test is None:
        print("[Error] x_test not available. Ensure data was extracted properly.")
        return

    errors = [i for i in range(len(self.y_true)) if self.y_pred[i] != self.y_true[i]]
    if not errors:
        print("✅ No misclassifications found!")
        return

    selected = random.sample(errors, min(num_samples, len(errors)))

    plt.figure(figsize=(10, 10))
    for i, idx in enumerate(selected):
        plt.subplot(3, 3, i + 1)
        plt.imshow(self.x_test[idx])
        plt.title(f"True: {self.class_names[self.y_true[idx]]}\nPred: {self.class_names[self.y_pred[idx]]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()
